In [6]:
import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os
from ipywidgets import interact, interactive_output
from utils import project_root, run_benchmark
import importlib


def title_widget(title):
    return widgets.HTML(value=f"<div style='font-size: 24px; margin-left: 12px'>{title}</div>")

benchmarks = sorted([
    os.path.splitext(filename)[0]
    for filename in os.listdir(os.path.join(project_root, 'benchmarks_code'))
    if filename not in ('__pycache__', '.ipynb_checkpoints')
])

def benchmark_widget(value, *args, **kwargs):
    return widgets.Dropdown(
        *args,
        options=benchmarks,
        value=value if value else benchmarks[0],
        description='Benchmark:',
        disabled=False,
        **kwargs
    )

def read_benchmark_results(benchmark):
    with open(f'benchmarks_results/{benchmark}.json') as f:
        results = json.loads(f.read())
    df = pl.DataFrame({'elapsed': results['elapsed']}).with_columns(
        (pl.col('elapsed') * 1000).alias('elapsed_ms')
    )
    del results['elapsed']
    return results, df

def agg_benchmark_results(meta, df):
    num_operations = meta['num_operations']
    return df.select(
        pl.col('elapsed_ms').median().alias('median (ms)'),
        pl.col('elapsed_ms').quantile(0.95).alias('q95 (ms)'),
    ).with_columns(
        pl.lit(num_operations).alias('num_ops'),
        (pl.col('median (ms)') / num_operations).alias('median_ms_per_op'),
        (pl.col('q95 (ms)') / num_operations).alias('q95_ms_per_op'),
    )

# save last selection so if we refresh, we don't have to keep re-selecting it
last_analyzed = None

In [10]:
display(title_widget('Benchmark Output'))
b_widget = benchmark_widget(last_analyzed)

def plot_benchmark(benchmark):
    global last_analyzed
    last_analyzed = benchmark

    try:
        meta, df = read_benchmark_results(benchmark)
    except:
        print('No data available')
        plt.clf()
        return
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # df = pl.read_csv('results/simple_ops.csv')
    plot = sns.kdeplot(x='elapsed_ms', data=df, ax=axes[0])
    plot.set_title(f'{benchmark} - Kernel Density Estimation')

    plt.figure()

    plot = sns.lineplot(x='index', y='elapsed_ms', data=df.with_row_index(), ax=axes[1])
    plot.set_title(f'{benchmark} - Elapsed Time')

    # plt.ylim(0)  # enable this to start y-axis at 0
    plt.show()

    print(agg_benchmark_results(meta, df))

def run_benchmark_and_plot(benchmark, output):
    with output:
        output.clear_output()
        print('running benchmark:', benchmark)
        run_benchmark(benchmark)
        plot_benchmark(benchmark)

out = interactive_output(plot_benchmark, {'benchmark': b_widget})

run_button = widgets.Button(description='Run Benchmark')
run_button.on_click(lambda _: run_benchmark_and_plot(b_widget.value, out))
top_bar = widgets.HBox([b_widget, run_button])

display(widgets.VBox([top_bar, out]))

HTML(value="<div style='font-size: 24px; margin-left: 12px'>Benchmark Output</div>")

In [8]:
display(title_widget('Benchmark Comparison'))

def plot_benchmark_kde_only(benchmark):
    if not benchmark:
        return
        
    try:
        meta, df = read_benchmark_results(benchmark)
    except:
        print('no data available')
        plt.clf()
        return
    
    plt.figure()
    plot = sns.kdeplot(x='elapsed_ms', data=df)
    plot.set_title(f'{benchmark} - Kernel Density Estimation')
    plt.show()
    print(agg_benchmark_results(meta, df))

def run_benchmark_and_compare(benchmark, output):
    print('running benchmark:', benchmark)
    run_benchmark(benchmark)
    with output:
        output.clear_output()
        plot_benchmark(benchmark)

def show_one():
    b_widget = benchmark_widget(None)

    out = interactive_output(plot_benchmark_kde_only, {'benchmark': b_widget})
    run_button = widgets.Button(description='Run Benchmark')
    run_button.on_click(lambda _: run_benchmark_and_compare(b_widget.value, out))

    control_panel = widgets.HBox([b_widget, run_button])
    return widgets.VBox([control_panel, out])

display(widgets.HBox([show_one(), show_one()]))

HTML(value="<div style='font-size: 24px; margin-left: 12px'>Benchmark Comparison</div>")